In [4]:
import os
import pandas as pd
from tqdm import tqdm
import re

In [5]:
# Open the dataset documents and store their data into a DataFrame
def load_himym_dataset():
    episodes_folder = os.path.join(os.getcwd(), "Datasets", "Sources", "HIMYM", "Episodes")
    dataframe_rows = []
    # Get number of documents and their names
    documents_n = len(os.listdir(episodes_folder))
    documents_names = os.listdir(episodes_folder)

    # Loop over documents
    for i in tqdm(range(documents_n)):
        filename = documents_names[i]
        # Open document
        file = open(os.path.join(episodes_folder, filename))
        episode_index = filename[:-4]
        # Loop over lines (= words)
        for line in file.readlines():
                dataframe_row = {
                    "episode": episode_index,
                    "line": line,
                }
                dataframe_rows.append(dataframe_row)
    # Build the dataframe from the words
    df = pd.DataFrame(dataframe_rows)
    return df

In [6]:
# Execute creation of dataset
himym_df = load_himym_dataset()
himym_df.head()
himym_df.count()

100%|████████████████████████████████████████████████████████████████████████████████| 139/139 [00:02<00:00, 51.57it/s]


episode    39284
line       39284
dtype: int64

In [7]:
def process_himym_dataset(df):
    df = df[~df['line'].str.startswith("[")]
    df = df[~df['line'].str.startswith("(")]
    df['line'] = df['line'].str.strip()
    df['line'] = df['line'].str.replace(r"\(.*\)","")
    df[['character', 'line']] = df['line'].str.split(":", 1, expand=True)
    df = df[~df['line'].isnull()]
    df = df.reset_index(drop=True)
    return df
    
himym_df = process_himym_dataset(himym_df)

In [8]:
himym_df.head(20)

,episode,line,character
0,01x01,"Kids, I'm going to tell you an incredible sto...",Narrator
1,01x01,Are we being punished for something?,Son
2,01x01,No,Narrator
3,01x01,"Yeah, is this going to take a while?",Daughter
4,01x01,"Yes. Twenty-five years ago, before I was dad...",Narrator
5,01x01,It was way back in 2005. I was twenty-seven j...,Narrator
6,01x01,Will you marry me.,Marshall
7,01x01,"Yes, perfect! And then you're engaged, you po...",Ted
8,01x01,"Got it. Thanks for helping me plan this out, ...",Marshall
9,01x01,"Dude, are you kidding? It's you and Lily! I'v...",Ted


In [9]:
# NOTE: May consider feeding one sentence and one Barney reply or multiple sentences encoded with one Barney reply
def get_barney(himym_df, level=2):
    dataframe_rows = []
    idxs_barney = himym_df[himym_df['character'] == 'Barney'].index
    for i in range(-1, -level-1, -1):
        for j in idxs_barney:
            dataframe_row = {
                "reply": himym_df['line'][j],
                "sentence": himym_df['line'][j+i],
            }
            dataframe_rows.append(dataframe_row)
    df = pd.DataFrame(dataframe_rows)
    return df
    
barney_df = get_barney(himym_df)

In [10]:
barney_df.head()

,reply,sentence
0,"hey, so you know how I've always had a thing...",What was I doing? Your Uncle Marshall was tak...
1,"Okay, meet me at the bar in fifteen minutes, ...","Hey, you wanna do something tonight?"
2,Where's your suit!? Just once when I say suit...,Hey.
3,It was a blazer!,I did that one time.
4,I see what this is about. Have you forgotten ...,"You know, ever since college it's been Marsha..."


In [15]:
barney_path = os.path.join(os.getcwd(), "Datasets", "Characters", "Barney")
if not os.path.exists(barney_path):
    os.makedirs(barney_path)
barney_df.to_csv(os.path.join(barney_path, "Barney.csv"))

Using custom data configuration default-9d6f8460ba1955be



Dataset csv downloaded and prepared to C:\Users\tonel\.cache\huggingface\datasets\csv\default-9d6f8460ba1955be\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


In [20]:
from datasets import load_dataset

barney_hg = load_dataset('csv', data_files=os.path.join(barney_path, "Barney.csv"))
barney_hg = barney_hg["train"].train_test_split(test_size=0.1)
print(barney_hg["train"][0])

Using custom data configuration default-9d6f8460ba1955be
Reusing dataset csv (C:\Users\tonel\.cache\huggingface\datasets\csv\default-9d6f8460ba1955be\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)



{'Unnamed: 0': 5500, 'reply': " Shannon?! Shannon, Shannon... No, don't remember any Shannon.", 'sentence': ' You said what, you got to drink!'}


In [1]:
from transformers import BlenderbotTokenizer, TFBlenderbotForConditionalGeneration
mname = 'facebook/blenderbot-400M-distill'
model = TFBlenderbotForConditionalGeneration.from_pretrained(mname)
tokenizer = BlenderbotTokenizer.from_pretrained(mname)

All model checkpoint layers were used when initializing TFBlenderbotForConditionalGeneration.

All the layers of TFBlenderbotForConditionalGeneration were initialized from the model checkpoint at facebook/blenderbot-400M-distill.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBlenderbotForConditionalGeneration for predictions without further training.


In [2]:
UTTERANCE = "Fuck you."
print("Human: ", UTTERANCE)
inputs = tokenizer([UTTERANCE], return_tensors='tf')
reply_ids = model.generate(**inputs)
print("Bot: ", tokenizer.batch_decode(reply_ids, skip_special_tokens=True)[0])

Human:  Fuck you.
Bot:   I know, right?  I was so mad.  I don't know what I would have done if it happened to me.


In [24]:
def preprocess_function(examples):
    inputs = [example for example in examples["sentence"]]
    targets = [example for example in examples["reply"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_barney_hg = barney_hg.map(preprocess_function, batched=True)

ValueError: Input is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="tf")

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_barney_hg["train"],
    eval_dataset=tokenized_barney_hg["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()